<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-family:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
Base Python: Comma Separated Value (CSV) Files
              
</p>
</div>

DS-NTL-010824  
<p>Phase 1: Topic 3.2</p>
<br>
<br>

<div align = "right">
<img src="images/flatiron-school-logo.png" align = "right" width="200"/>
</div>


# Data Analysis with CSVs

In this lesson, we'll complete a full data analysis with a raw data file. Comma Separated Values files, known as CSVs, are one of the most common file formats for storing tabular data. In this lesson, we'll show you how to load one into memory and work with it using Python.

# Objectives

Perform a complete data analysis by...

- Creating a Python data structure from a .csv file
- Exploring and cleaning the data 
- Conducting descriptive analysis
- Visualizing the results

#### Comma Separated Value Format

- Tabular data
- Column entries separated by commas
- Typical file extensions: .csv, .dat, .txt
- Very common data format

Example: Track times (s) for 100m dash for four athletes across 3 meets.

# Loading a CSV

The before you can work with a CSV file, you need to load it into memory.

The before you can work with a CSV file, you need to load it into memory.

We'll be working with food data! Data source: https://www.kaggle.com/datasets/openfoodfacts/world-food-facts (version we're using has been lightly cleaned and edited for clarity, and because we don't need all 163 columns)

## Get the File Path

Make sure you have the path to your data file. For this example, it is in the root folder of this repository.

In [ ]:
# This will only work on Mac, Linux, or a separate bash terminal in Windows

!ls

We use `.` to indicate the root folder of the repository when working within this notebook.

In [ ]:
csv_file_path = './data/OpenFoodFacts.csv'

## Inspecting CSV files

In a CSV file, each line represents one row of tabular data, and consecutive values in that row are separated by a comma. Often, the first row contains the column names separated by commas, also known as field names. Let's confirm that this is the case and learn about our dataset by printing the first two lines of the file.


We can inspect the data directly in our terminal! Use the bash command head with the flag -n 5 to check out those first two rows.



### Printing Lines in Python

We use the `with open()` syntax to easily open and read the file in the notebook. Using this syntax will automatically close the file once the statement is done running.

In [ ]:
# This code prints the first line of the CSV file

with open(csv_file_path) as csvfile:
    print(csvfile.readline())

Next, we can print the second line to look at an example of one observation from our dataset. This represents data form one product.

In [ ]:
# This code prints the second line of the CSV file

with open(csv_file_path) as csvfile:
    csvfile.readline()
    print(csvfile.readline())

### Printing Lines in bash

Alternatively, you could use a bash command to print the first two lines.

In [ ]:
# This will only work on Mac, Linux, or a separate bash terminal in Windows

!head -n 2 ./data/OpenFoodFacts.csv

In [ ]:
import pandas as pd
facts = pd.read_csv('data/OpenFoodFacts.csv')

In [ ]:
facts.info()

### Discussion
Based on these two lines, what can we infer about the contents of the data file? What questions arise for you about the data?

## Using the `csv` module

The [`csv` module](https://docs.python.org/3/library/csv.html) lets us easily process data in CSV files. We will use it to read each row in the `.csv` file and store its information in a Python object we can use for further analysis. 

In [ ]:
import csv


The [`csv.DictReader`](https://docs.python.org/3/library/csv.html#csv.DictReader) object is a file reader, reading each row and then converting it to a dictionary. By default, it turns each row into a dictionary, using the field names from the first row as the keys.

In [ ]:
# Print OrderedDict from first row of CSV file 

with open(csv_file_path) as csvfile:
    reader = csv.DictReader(csvfile)
    print(next(reader))

Let's get all of the data out of our file and into dictionaries, and store those dictionaries in a new list called `products`.

In [ ]:
products = []

with open(csv_file_path) as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        products.append(row)

In [ ]:
# first 3 entries
products[0:3]

# Data Preparation

Now that we've gotten all of our data into a Python object, we can prepare it for analysis. Let's look at a sample observation and consider how we might want to process it to make it easier to analyze.

In [ ]:
products[0]

**Discussion:** How might you clean these data to make them easier to analyze?

## Clean the Serving Size (and other numbers)

You might have noticed that all of these numbers are strings! Hard to do things like math on strings.

In addition, the `serving_size` here has ` g` at the end - in fact, I know from doing more analysis on this data that ALL of these are in grams, and have ` g` at the end. Let's remove that so we can treat those as numbers too!

In [ ]:
# First, let's do a one off example - how to we turn a string into a float?
float(products[0]['sugars_100g'])

In [ ]:
# And how can we remove the ' g' from the end of the serving sizes?
float(products[0]['serving_size'].strip(" g"))

In [ ]:
# Now that we've figured out what to do, let's clean up these dicts!
for product in products:
    # Clean up serving size
    product['serving_size_g'] = float(product['serving_size'].strip(" g"))

    # Clean up all the values if the key contains "100g"
    for key, detail in product.items():
        if '100g' in key:
            # now - we have some blanks in here!
            # introducing: try / except!
            try: # tries to do this first thing
                product[key] = float(detail)
            except: # does this if the first thing doesn't work
                product[key] = 0.0

In [ ]:
# Check it out:
products[0]

## Clean the Dates

Next we'll clean the dates so that we can easily get the month and year when each food was added to this database.

### Using Python built-in methods

**Activity**: Process the `products` list to add numeric values for the month and year when each entry was first created! (aka parse out `created_date`)

<details>
    <summary><b><u>Answer</u></b></summary>
   
   
    
```python    
for product in products:
    # Save the date to a variable
    string_date = product["created_date"]

    # Extract the sale year and month from the string, and cast to int
    created_year = int(string_date[0:4])
    created_month = int(string_date[5:7])    

    # Add the sale year and month to each dictionary element of sales
    product["created_year"] = created_year
    product["created_month"] = created_month
```
</details>

In [ ]:
products[0]

# Data Analysis

Now that you have a cleaned list with the property sales data, we can finally start to perform some data analysis.

## Question 1: What is the range of years for which we have data?

<details>
            <summary><b><u>Answer</u></b></summary>

```python
years = [product['created_year'] for product in products]
print(min(years))
print(max(years))
```    
</details>

## Question 2: What is the average number of calories per 100g for all of these foods?

In [ ]:
## Your work here


<details>
            <summary><b><u>Answer</u></b></summary>

```python
cals = [product['energy_100g'] for product in products]

sum(cals) / len(cals)
```
</details>

## Question 3: What is the average amount of sugar per 100g in foods that are more than 1000 calories per 100g?

In [ ]:
## Your work here

<details>
            <summary><b><u>Answer</u></b></summary>

```python
prods_over_1000cal = [product for product in products if product['energy_100g'] > 1000]
sugar = [product['sugars_100g'] for product in prods_over_1000cal]
sum(sugar) / len(sugar)
```
</details>

## Question 4: How many foods were added (created) in each year?

In [ ]:
## Your work here


<details>
            <summary><b><u>Answer 1</u></b></summary>

```python
prod_year_count = {}
list_created_years = [product['created_year'] for product in products]
unique_prod_years = set(list_created_years)
for unique_year in unique_prod_years:
    num_added = len([year for year in list_created_years if year == unique_year])
    prod_year_count[unique_year] = num_added
    
prod_year_count
```
</details>

<details>
            <summary><b><u>Answer 2</u></b></summary>

```python
food = {}

for i in products:
    if i['created_year'] not in food:
        food[i['created_year']] = 1
    elif i['created_year'] in food:
        food[i['created_year']] += 1

food
```
</details>

# Chart the Data

This rendering of the data directly in the notebook is helpful, but it takes a lot of effort to read it and make sense of the trends - let's visualize the results to make this easier.

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# create a figure and one plot
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 5))

# place data in a bar chart
# where the x-axis is each year and
# the y-axis is the number of products added per year
ax.bar(prod_year_count.keys(),
       prod_year_count.values())

# set axis labels
ax.set_xlabel("Year")
ax.set_ylabel("Number of Products Added")

# give the bar chart a title
ax.set_title("Foods Added to the Open Food Facts Database")

# display bar chart clearly
fig.tight_layout()

# Level Up: `Counter`

The `collections` package has a lot of useful tools for working with Python objects containing multiple elements. You can use the `Counter` class to easily count the number of values in a list.

For example, let's say that for all dictionaries in `sales`, we want to count how many property sales occured in each year. 

In [ ]:
from collections import Counter

prod_year_count = Counter([product['created_year'] for product in products])
prod_year_count

Let's sort these by year. Since the value in `sale_year` is an integer, we can use Python's built-in [`sorted`](https://docs.python.org/3/library/functions.html#sorted) function to transform the `Counter` object into a `list` of `tuples` - `(year, count)` which are sorted in ascending order by year.

In [ ]:
prod_year_count = sorted(prod_year_count.items())
prod_year_count

# Level Up: Using `datetime`

Python has a [`datetime`](https://docs.python.org/3.7/library/datetime.html) package that is the standard tool for handling dates and times. `datetime` objects make it easy to do fun things like subtract dates to calculate how far apart they are.

In [ ]:
from datetime import datetime

In [ ]:
for product in products:
    # Transform DocumentDate from string to datetime
    product["clean_date"] = datetime.strptime(product["last_modified_date"], 
                                           "%Y-%m-%d")
    
    # Add the sale year and month to each dictionary element of sales
    product["last_modified_year"] = product["clean_date"].year
    product["last_modified_month"] = product["clean_date"].month

Let's inspect our work

In [ ]:
products[0]